## Evaluation of model performances

This script does not require GPUs and can be run directly from Jupyter Notebook. However, the default file paths are currently set for execution on Google Colab. Change these variables appropriately, when/if required.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os, sys
from collections import OrderedDict
import pandas as pd
import scipy.stats as st

In [ ]:
# navigate to results directory
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# specify path to baseline and tcrp performance results
%cd /content/drive/MyDrive/MBP1413H/tcrp_model/data/results
dataset = 'GDSC'
baseline_path = Path(f"{dataset}/baseline_performances/")
tcrp_path = Path(f"{dataset}/TCRP_performances/")

In [ ]:
# parse TCRP results 
%%time

results = {}
for outer_directory in tcrp_path.glob("*"): 
    drug = outer_directory.stem
    results[drug] = {}

    for inner_directory in outer_directory.glob("*"): 
        tissue = inner_directory.stem
        results[drug][tissue] = {}
        if str(inner_directory).split("/")[-1] != ".ipynb_checkpoints":
          data = np.load(inner_directory / "TCRP_performance.npz")

          for model in ['TCRP']: 
              zero = data[f"{model}-zero"]
              new_data = data["TCRP-fewshot"] 
              fewshot = np.vstack([new_data for _ in range(10)])
              zero = np.vstack([zero for _ in range(10)]) # There is only 1 possible zero-shot, so expanding for all trials
              performance = np.mean(np.hstack([zero, fewshot]), axis=0)
              results[drug][tissue][model] = performance    


In [ ]:
# parse baseline results 
%%time

for outer_directory in baseline_path.glob("*"): 
    drug = outer_directory.stem
    
    for inner_directory in outer_directory.glob("*"): 
        tissue = inner_directory.stem
        data = np.load(inner_directory / "baseline_performance.npz")
        
        for model in ['Linear Regression', 'Nearest Neighbour', 'Random Forest','Neural Network']: 
          zero = data[f"{model}-zero"]
          zero = np.vstack([zero for _ in range(20)]) # There is only 1 possible zero-shot, so expanding for all trials
          performance = np.median(np.hstack([zero, data[f"{model}-fewshot"]]), axis=0)
            
          results[drug][tissue][model] = performance    

In [ ]:
for i in results:
    remove_key = results[i].pop(".ipynb_checkpoints", None)
results

In [ ]:
results_by_baseline = {'Linear Regression': [], 'Nearest Neighbour': [], 'Random Forest': [],'Neural Network': [], 'TCRP': []}

# here, you can choose to analyze a single drug and/or tissue 
drug = "GSK429286A"
tissue = "liver"
res_drug = dict(results[drug])
res_drug_tissue = res_drug[tissue]

'''
# uncomment this to analyze all drugs and tissues
for drug, d in res.items(): 
    for tissue, d in d.items(): 
        for model, p in d.items(): 
            results_by_baseline[model].append(p)
'''

'''
# uncomment this to analyze all tissues for the specified drug
for tissue, d in res_drug.items():
    for model, p in d.items(): 
        results_by_baseline[model].append(p)
'''

#'''
# uncomment this to analyze the specified tissue and specified drug          
for model, ps in res_drug_tissue.items(): 
    results_by_baseline[model] = np.vstack(ps)

#'''


In [ ]:
def get_statistics(data): 
    median = np.nanmean(data, axis=0)
    stdev = np.nanstd(data, axis=0)
    index = np.random.choice(data.shape[0], size=(data.shape[0], 1000), replace=True)
    resampled = np.nanmean(data[index], axis=0)

    low = np.nanpercentile(resampled, 2.5, axis=0)
    high = np.nanpercentile(resampled, 97.5, axis=0)
    
    ci = np.vstack([median - low, high-median])
    
    return median, stdev, ci

In [ ]:
fig, ax = plt.subplots()
x = np.arange(11)
kwargs = {'capsize': 4}

color_dict={"TCRP":"red",
            "Random Forest":"turquoise",
            "Neural Network":"purple",
            "Nearest Neighbour": "navy",
            "Linear Regression":"yellowgreen"}

for model, ps in results_by_baseline.items(): 
    median, std, yerr = get_statistics(ps)
    ax.errorbar(x, median, yerr=yerr, label=model, color=color_dict[model],marker='o', 
                ms = 3, mfc = "black", mec ="black", lw = 1,**kwargs)

ax.legend()
labels = ['Pretrained'] + [str(i) for i in range(1, 11)]
ax.set_xticks(x)
ax.set_xticklabels(labels)

#ax.set_ylim([0, 0.3])
ax.set_xlabel("Number of samples, few-shot learning")
ax.set_ylabel("Correlation (predicted, actual)")
ax.set_title("Transfer of predictive models across tissue types")
ax.legend(loc='upper right', bbox_to_anchor=(1.45, 1.03), numpoints=1)

In [ ]:
fig, ax = plt.subplots()
x = np.arange(11)
kwargs = {'capsize': 4}

color_dict={"TCRP":"red",
            "Random Forest":"turquoise",
            "Neural Network":"purple",
            "Nearest Neighbour": "navy",
            "Linear Regression":"yellowgreen"}

plt_dict = {}

vec = ["TCRP","Random Forest","Neural Network","Nearest Neighbour","Linear Regression"]
for model1 in vec:
  for model2, ps in results_by_baseline.items(): 
    median, std, yerr = get_statistics(np.array(ps))
    if(model1 == model2):
      ax.bar(model1, median[0], color=color_dict[model1])

ax.legend()

#ax.set_ylim([0, 0.3])
plt.xticks(rotation=20)
ax.set_ylabel("Correlation (predicted, actual)")
ax.set_title("Transfer of predictive models across tissue types\nChallenge 1b")
ax.legend(loc='upper right', bbox_to_anchor=(1.45, 1.03), numpoints=1)

In [ ]:
import itertools

flipped_results = dict([(x, dict([(k, results[k][x]) for k,v in results.items() if x in results[k]])) 
for x in set(itertools.chain(*[z for z in results.values()]))])

In [ ]:
def prepare_points(model_dict):
  items = model_dict.items()
  myList = (items) 
  y,x = zip(*myList) 
  return y,x

In [ ]:
TCRP_tissue = {}
linear_tissue = {}
RF_tissue = {}
KNN_tissue = {}
NN_tissue = {}

for tissue, d in flipped_results.items(): # key = tissue; value = performance of prediction for each drug
  for drug, r in d.items(): 
    TCRP_tissue[tissue] = np.mean(r["TCRP"])
    linear_tissue[tissue] = np.mean(r["Linear Regression"])
    RF_tissue[tissue] = np.mean(r["Random Forest"])
    NN_tissue[tissue] = np.mean(r["Neural Network"])
    KNN_tissue[tissue] = np.mean(r["Nearest Neighbour"])


In [ ]:
{k: v for k, v in sorted(TCRP_tissue.items(), key=lambda item: item[1])}

In [ ]:
TCRP_tissue = {k: v for k, v in sorted(TCRP_tissue.items(), key=lambda item: item[1])}   
linear_tissue = dict(OrderedDict((k, linear_tissue[k]) for k in list(TCRP_tissue.keys())))
NN_tissue = dict(OrderedDict((k, NN_tissue[k]) for k in list(TCRP_tissue.keys())))
RF_tissue = dict(OrderedDict((k, RF_tissue[k]) for k in list(TCRP_tissue.keys())))
KNN_tisse = dict(OrderedDict((k, KNN_tissue[k]) for k in list(TCRP_tissue.keys())))

In [ ]:
TCRP_y,TCRP_x = prepare_points(TCRP_tissue)

linear_y,linear_x = prepare_points(linear_tissue)
RF_y,RF_x = prepare_points(RF_tissue)
KNN_y,KNN_x = prepare_points(KNN_tissue)
NN_y,NN_x = prepare_points(NN_tissue)

In [ ]:
fig = plt.figure()


fig, ax = plt.subplots()

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
#fig.set_size_inches(8,12)

RF = plt.scatter(RF_x,RF_y,color="deepskyblue",s=100)
RF.set_label("Random Forest")
linear = plt.scatter(linear_x, linear_y,color="yellowgreen",s=100)
linear.set_label("Linear Regression")
KNN = plt.scatter(KNN_x, KNN_y,color="dodgerblue",s=100)
KNN.set_label("Nearest Neighbours")
NN = plt.scatter(NN_x,NN_y,color="mediumpurple",s=100)
NN.set_label("Neural Network")
TCRP = plt.scatter(TCRP_x,TCRP_y,color="crimson",s=100)
TCRP.set_label("TCRP")
plt.xlabel("Correlation (predicted, actual)")
plt.xlim(-0.5,0.8)
plt.xticks([-0.5,-0.3,-0.1,0.1,0.3,0.5,0.7])
#plt.tight_layout()
plt.legend(loc='upper right', bbox_to_anchor=(1.45, 1.03))
plt.title("Transfer of predictive models across tissue types")